import pickle
import os
import csv
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import *
from tqdm import tqdm
import re
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import umap
from sklearn.decomposition import PCA

## Load TransformerSentence class

In [2]:
class TransformerSentence():
    def __init__(self, sentence_str,
                 model=BertModel.from_pretrained('scibert-scivocab-uncased'), 
                 tokenizer=BertTokenizer.from_pretrained('scibert-scivocab-uncased')):
        
        self.raw_string = sentence_str
        self.model = model
        self.tokenizer = tokenizer
        self.summary = {}

        
    def write_summary(self, input_tokens=None, 
                      hidden_states=None, 
                      hidden_attentions=None,
                      print_tokens=True):
        
        if (input_tokens or hidden_states or hidden_attentions) is None:
            input_tokens, hidden_states, hidden_attentions = self.forward()
        
        # this replaces adds a "_{counter}" to the repreated tokens, so that 
        # they can be used uniquely as the keys for the embeddings dictionary
        input_tokens = TransformerSentence.make_unique(input_tokens)
        
        if print_tokens:
            print('Sentence Tokenization: ', input_tokens)
            
        # write summary into the object
        self.summary['input_tokens'] = input_tokens
        self.summary['states'] = hidden_states
        self.summary['attentions'] = hidden_attentions

        self.summary['token_embeddings'] = {input_token: hidden_states[:, i, :] 
                                            for i, input_token in enumerate(input_tokens)}
        
    def forward(self):
        encoded_inputs_dict = self.tokenizer.encode_plus(self.raw_string)
        input_ids = encoded_inputs_dict['input_ids']
        input_tensor = torch.tensor([input_ids])
        input_tokens = [self.tokenizer.decode(input_ids[j]).replace(' ', '') 
                        for j in range(len(input_ids))]
        
        final_attention, final_state, hidden_states_tup, hidden_attentions_tup = self.model(input_tensor)
        
        # stacking states and attentions along the first dimention (which corresponds to the batch when necessary)
        hidden_attentions = torch.cat(hidden_attentions_tup, dim=0) # 'layers', 'heads', 'queries', 'keys'
        hidden_states = torch.cat(hidden_states_tup, dim=0) # 'layers', 'tokens', 'embeddings'
        
        return input_tokens, hidden_states.detach(), hidden_attentions.detach()
    
    
    def attention_from_tokens(self, token1, token2, display=True):
        input_tokens = self.summary['input_tokens']
        
        if (token1 and token2) not in input_tokens:
            raise ValueError('One or both of the tokens introduced are not in the sentence!')
            
        idx1, idx2 = input_tokens.index(token1), input_tokens.index(token2)
        attention = self.summary['attentions'][:, :, idx1, idx2].numpy()
        if display:
            TransformerSentence.display_attention(attention, title=(token1, token2))
        return attention
    
    
    def attention_from_idx(self, i, j, display=True):
        attention = self.summary['attentions'][:, :, i, j].numpy()
        if display:
            TransformerSentence.display_attention(attention, title=f'Token idx: {(i, j)}')
        return attention
    
    def visualize_token_path(self, fit, 
                             tokens_to_follow=None, 
                             print_tokens=False, 
                             fig_axs=(None, None), 
                             figsize=(10, 10)):
        
        if tokens_to_follow is None:
            all_tokens = self.summary['input_tokens']
            regex = re.compile(r'^[a-zA-Z]')
            tokens_to_follow = [i for i in all_tokens if regex.search(i)]
            
        if print_tokens: print(tokens_to_follow)  
            
        colors = list(range(len(tokens_to_follow)))
        projections = []
        layer_depth = self.summary['states'].size()[0]
        
        for i in range(layer_depth):
            layer_embeddings = self.summary['states'][i, :, :]
            projection = fit.transform(layer_embeddings)
            projections.append(projection)

        data = np.stack(projections, axis=0)
        if None in fig_axs:
            fig, axs = plt.subplots(figsize=figsize)
        for token in tokens_to_follow:
            i = self.summary['input_tokens'].index(token)
            plt.plot(data[:,i,0], data[:,i,1], '-o', alpha=0.3)
            plt.annotate(s=token, xy=(data[0, i, 0], data[0, i, 1]))

        plt.show()
        
    def visualize_sentence_shape(self, fit, tokens_to_follow=None, 
                                 print_tokens=False, 
                                 fig_axs=(None, None), 
                                 figsize=(10, 10)):

        if tokens_to_follow is None:
            all_tokens = self.summary['input_tokens']
            regex = re.compile(r'^[a-zA-Z]')
            tokens_to_follow = [i for i in all_tokens if regex.search(i)]

        if print_tokens: print(tokens_to_follow)  

        colors = list(range(len(tokens_to_follow)))
        projections = []
        layer_depth = self.summary['states'].size()[0]
        
        # get list of indeces of the tokens to follow
        idxs = [self.summary['input_tokens'].index(token) for token in tokens_to_follow] 
        token_embeddings = self.summary['states'][-1, idxs, :]
        data = fit.transform(token_embeddings)
        
        if None in fig_axs:
            fig, axs = plt.subplots(figsize=figsize)
            
        plt.plot(data[:,0], data[:,1], '-o')
        for i, token in enumerate(tokens_to_follow):
            plt.annotate(s=token, xy=(data[i, 0], data[i, 1]))
        #plt.show()
    
    
    def save(self, name, path='.'):
        with open(os.path.join(path, name), 'wb') as file:
            pickle.dump(self, file)
    
    
    @staticmethod
    def visualize_embedding(embedding, title=None, vmax=None, vmin=None):
        if (vmax or vmin) is None:
            vmax = max(embedding)
            vmin = min(embedding)
            
        N = embedding.size()[0]
        h = math.ceil(math.sqrt(N))
        # N = a*b where abs(a-b) is minimum
        while (N % h != 0):
            h -= 1
        w = int(N / h)
        visualization = embedding.reshape((h, w)).numpy()
        fig, ax = plt.subplots()
        im = ax.imshow(visualization, vmax=vmax, vmin=vmin, cmap='viridis')
        fig.colorbar(im)
        if title is not None:
            ax.set_title(title)
        plt.show()
    
    @staticmethod
    def display_attention(attention, title=None):
        fig, ax = plt.subplots()
        im = ax.imshow(attention, vmin=0., vmax=1., cmap='viridis')
        fig.colorbar(im)
        if title is not None:
            ax.set_title(title)
        ax.set_xlabel('HEADS')
        ax.set_ylabel('LAYERS')
        plt.show()
    
    @staticmethod
    def load(name, path='.'):
        with open(os.path.join(path, name), 'rb') as file:
            SentenceObject = pickle.load(file)
        return SentenceObject
    
    @staticmethod
    def make_unique(L):
        unique_L = []
        for i, v in enumerate(L):
            totalcount = L.count(v)
            count = L[:i].count(v)
            unique_L.append(v + '_' + str(count+1) if totalcount > 1 else v)
        return unique_L

## Load pre-trained models and tokenizers

In [3]:
# Preloading models (this is the most costly)
BertBaseModel = BertModel.from_pretrained('bert-base-uncased')
BertBaseTokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
BertLargeModel = BertModel.from_pretrained('bert-large-uncased')
BertLargeTokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
SciBertModel = BertModel.from_pretrained('scibert-scivocab-uncased')
SciBertTokenizer = BertTokenizer.from_pretrained('scibert-scivocab-uncased')
SciBertBaseVocabModel = BertModel.from_pretrained('scibert-basevocab-uncased')
SciBertBaseVocabTokenizer = BertTokenizer.from_pretrained('scibert-basevocab-uncased')

## Load quora questions dataset

In [4]:
# dataset loader function
def load_dataset(txt_path="../datasets/quora_questions.txt", 
                 MODEL=SciBertModel,
                 TOKENIZER=SciBertTokenizer):
    
    # Read input sequences from .txt file and put them in a list
    with open(txt_path) as f:
        text = f.read()
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    try:
        sentences.remove('') # remove possible empty strings
    except:
        None
    
    list_SentenceObj, ALL_INITIAL_EMBEDDINGS, ALL_CONTEXT_EMBEDDINGS = [], [], []
    
    for raw_sentence in tqdm(sentences):
        SentenceObj = TransformerSentence(raw_sentence,
                                          model=MODEL,
                                          tokenizer=TOKENIZER)
        SentenceObj.write_summary(print_tokens=False)
        list_SentenceObj.append(SentenceObj)
        ALL_INITIAL_EMBEDDINGS.append(SentenceObj.summary['states'][0, :, :])
        ALL_CONTEXT_EMBEDDINGS.append(SentenceObj.summary['states'][-1, :, :])

    ALL_INITIAL_EMBEDDINGS = torch.cat(ALL_INITIAL_EMBEDDINGS, dim=0)
    ALL_CONTEXT_EMBEDDINGS = torch.cat(ALL_CONTEXT_EMBEDDINGS, dim=0)
    
    return list_SentenceObj, ALL_INITIAL_EMBEDDINGS, ALL_CONTEXT_EMBEDDINGS

In [5]:
scibert_sentences, SCI_EMB_i, SCI_EMB_c = load_dataset(MODEL=SciBertModel, 
                                                       TOKENIZER=SciBertTokenizer)
bert_sentences, BASE_EMB_i, BASE_EMB_c = load_dataset(MODEL=BertBaseModel, 
                                                      TOKENIZER=BertBaseTokenizer)

100%|██████████| 301/301 [00:44<00:00,  6.74it/s]


In [6]:
for i, sentence in enumerate(bert_sentences):
    print(i, sentence.raw_string)

0 Are convolutional neural networks useful for tasks other than image classification?
1 Are non-causal temporal convolutions the equivalence of Bi-LSTM?
2 Are there any techniques, other than RNN/LSTM, to handle time series data?
3 Bayesian Inference: What is a Dirichlet process in layman's terms?
4 Can an objective account of statistical inference be based on frequentist methods and Bayesian methods?
5 Can Bayesian Network be combined with Deep Learning and/or reinforcement Learning to figure out causation?
6 Can deep neural networks learn the minimum function?
7 Can facial recognition tools be fooled?
8 Can recurrent neural networks with LSTM be used for time series prediction?
9 Can RNN be used for time series dynamic modelling?
10 If yes, how?
11 Can you explain the HMM algorithm?
12 Classification (machine learning): When should I use a K-NN classifier over a Naive Bayes classifier?
13 Computer Vision: What is the difference between HOG and SIFT feature descriptor?
14 Computer Vis

In [7]:
def save_data(TENSORS, LABELS, PATH):
    if type(TENSORS) is not np.ndarray:
        TENSORS = TENSORS.numpy
    
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    
    np.savetxt(os.path.join(PATH, 'tensors.tsv'), TENSORS, delimiter='\t', newline='\n')
    with open(os.path.join(PATH, 'labels.tsv'), 'w') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\n')
        tsv_output.writerow(LABELS)

In [8]:
def get_tensors_and_labels(sentences_list, sentence_indices, layer):
    sentences = [sentences_list[i] for i in sentence_indices]
    TENSORS, LABELS = [], []
    for i, sentence in enumerate(sentences):
        t = sentence.summary['states'][layer, :, :].numpy()
        L = sentence.summary['input_tokens']
        idx = sentence_indices[i]
        L = [f'{Li} \t {idx:03d}' for Li in L ]
        TENSORS.append(t)
        LABELS.extend(L)
    
    LABELS.insert(0, f'token \t sentence_idx')    
    TENSORS = np.concatenate(TENSORS, axis=0)
    
    return TENSORS, LABELS

## Get the embeddings from sentences and save them as .tsv

In [10]:
indices = [7, 35, 97, 118]
sentences = [scibert_sentences[i] for i in indices]
PATH = '/Users/sergicastellasape/Repos/zeta-alpha/data/embeddings/scibert/contextual-7-35-97-118'
TENSORS, LABELS = get_tensors_and_labels(scibert_sentences, indices, -1)
save_data(TENSORS, LABELS, PATH)